In [1]:
import numpy as np
import scipy as sp

# Define Hamiltonian
We want a single body Hamiltonian as a $d \times d$ matrix. We can either choose a specific Hamiltonian (TFI, AKLT, cluster state) or a random Hamiltonian.

In [28]:
# Random Hamiltonian
d = 10
H = np.random.rand(d,d) + 1.j * np.random.rand(d,d)
H = H + H.conj().T

In [3]:
# Periodic TFI
def kron(ops):
    op = 1
    for o in ops:
        op = np.kron(op, o)
    return op

Z = np.array([[1,0],[0,-1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1.j],[1.j,0]])
Id = np.eye(2)

def build_TFI_Hamiltoinan(L, Jz, hx, hz):
    # Build full 2^L x 2^L Hamiltonian

    H = np.zeros((2**L, 2**L), dtype=np.float64)

    for i in range(L):
        H += hx * kron([Id]*i + [X] + [Id]*(L-i-1))
        H += hz * kron([Id]*i + [Z] + [Id]*(L-i-1))

    Ising = [Z, Z] + [Id] * (L-2)
    for i in range(L):
        H += Jz * kron(Ising)
        # Shift the first element to the last; periodic BCs!
        Ising.append(Ising.pop(0))
    return H
    
def build_cluster_state_Hamiltoinan(L):
    # Build full 2^L x 2^L Hamiltonian

    H = np.zeros((2**L, 2**L), dtype=np.float64)

    Cluster = [Z, X, Z] + [Id] * (L-3)
    for i in range(L):
        H += -1 * kron(Cluster)
        # Shift the first element to the last; periodic BCs!
        Cluster.append(Cluster.pop(0))
    return H

#H = build_TFI_Hamiltoinan(5, 1, -1.4, 0.9045)
#H = build_cluster_state_Hamiltoinan(5)
#H = Z
d = H.shape[0]

In [29]:
L, U = np.linalg.eigh(H)
assert np.isclose(np.linalg.norm(U @ np.diag(L) @ U.conj().T - H), 0.0)

In [30]:
print(L)

[-2.48803285 -1.62378432 -1.29884051 -0.35736266 -0.23994722  0.55064579
  1.4655626   1.80512199  2.19963017  9.43507219]


# Define Kraus operators for discrete quantum channel - Block Encoding

In [31]:
L_shift = L - np.min(L)
print(L_shift)
H_shift = U @ np.diag(L_shift) @ U.conj().T

[ 0.          0.86424853  1.18919234  2.13067019  2.24808563  3.03867864
  3.95359545  4.29315484  4.68766302 11.92310504]


In [32]:
beta = 0.1
gamma_1 = sp.linalg.expm(-beta * H_shift)
assert np.isclose(np.linalg.norm(U @ sp.linalg.expm(-beta * np.diag(L_shift)) @ U.conj().T - gamma_1), 0.0)

In [33]:
gamma_2 = U @ np.diag(np.sqrt(1 - np.exp(-2*beta*L_shift))) @ U.conj().T

In [34]:
P = np.roll(np.eye(d), 1, axis=1)
P_gamma_2 = P @ gamma_2

In [35]:
assert np.isclose(np.linalg.norm(np.eye(d) - (gamma_1.conj().T @ gamma_1 + gamma_2.conj().T @ gamma_2)), 0.0)
assert np.isclose(np.linalg.norm(np.eye(d) - (gamma_1.conj().T @ gamma_1 + P_gamma_2.conj().T @ P_gamma_2)), 0.0)

In [36]:
assert np.isclose(np.linalg.norm(gamma_1 - gamma_1.conj().T), 0.0)
assert np.isclose(np.linalg.norm(gamma_2 - gamma_2.conj().T), 0.0)
assert not np.isclose(np.linalg.norm(P_gamma_2 - P_gamma_2.conj().T), 0.0)

Both $\Gamma_1$ and $\Gamma_2$ are Hermitian, as they should be. Both are matrix exponentials of Hermitian matrices with a real exponent (imaginary time evolution and all). Additionally, we have checked that $\sum_i K_i^\dagger K_i = 1$. $P \Gamma_2$ is no longer Hermitian since we have applied a permutation matrix.

# Define Kraus operators for discrete quantum channel - Weak Measurement
Operators from https://arxiv.org/pdf/2202.09100 without the conditional correction.

In [79]:
L_balanced = L - (np.min(L) + np.max(L))/2
print(L_balanced)
H_balanced = U @ np.diag(L_balanced) @ U.conj().T

[-5.96155252 -5.09730399 -4.77236017 -3.83088233 -3.71346689 -2.92287387
 -2.00795706 -1.66839767 -1.2738895   5.96155252]


In [102]:
gamma = 0.1
gamma = (-np.pi/4 + 1.e-8) / np.min(L_balanced)
#gamma = np.pi/4
assert np.min(L_balanced) * gamma >= -np.pi/4
assert np.max(L_balanced) * gamma <= np.pi/4
print(np.min(L_balanced) * gamma, np.max(L_balanced) * gamma) 

-0.7853981533974482 0.7853981533974481


In [103]:
M_0 = 1/np.sqrt(2) * (sp.linalg.cosm(gamma * H_balanced) - sp.linalg.sinm(gamma * H_balanced))
M_1 = 1/np.sqrt(2) * (sp.linalg.cosm(gamma * H_balanced) + sp.linalg.sinm(gamma * H_balanced))

In [104]:
assert np.isclose(np.linalg.norm(np.eye(d) - (M_0.conj().T @ M_0 + M_1.conj().T @ M_1)), 0.0)

# Define vectorized channel
We want the superoperator $E = \sum_i K_i \otimes \overline{K_i}$, given Kraus operators $K_i$.

In [105]:
E = np.kron(gamma_1, gamma_1.conj()) + np.kron(gamma_2, gamma_2.conj())
print(E.shape)

(100, 100)


In [106]:
P_E = np.kron(gamma_1, gamma_1.conj()) + np.kron(P_gamma_2, P_gamma_2.conj())
print(P_E.shape)

(100, 100)


In [107]:
M_E = np.kron(M_0, M_0.conj()) + np.kron(M_1, M_1.conj())
print(M_E.shape)

(100, 100)


In [108]:
assert np.isclose(np.linalg.norm(E - E.conj().T), 0.0)
assert not np.isclose(np.linalg.norm(P_E - P_E.conj().T), 0.0)
assert np.isclose(np.linalg.norm(M_E - M_E.conj().T), 0.0)

The superoperator without permutation is Hermitian. This makes sense since all Kraus operators are. However, the superoperator isn't.

In [109]:
L2, U2 = np.linalg.eigh(E)
assert np.isclose(np.linalg.norm(E - U2 @ np.diag(L2) @ U2.conj().T), 0.0)
#assert np.isclose(np.linalg.norm(E - U2 @ np.diag(L2) @ sp.linalg.inv(U2)), 0.0)

Columns of $U2$ are the right eigenvectors. I am using `np.linalg.eig` rather than `np.linalg.eigh` since the fixed point eigenvectors for `eigh` aren't Hermitian when viewed as density matrices. I am not sure why this is the case.

In [110]:
print(np.max(L2))
L2_index = np.where(np.isclose(np.abs(L2), 1.0, atol=1.e-14, rtol=1.e-14))[0]
print(L2_index, len(L2_index))

1.000000000000001
[90 91 92 93 94 95 96 97 98 99] 10


In [111]:
L2_sort = np.abs(L2)
L2_sort.sort()
print(L2_sort[-len(L2_index)-1:])

[0.99987451 1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.        ]


Why are there multiple ($d$ for random Hamiltonian) fixed points? I guess there isn't a unique fixed point. This is because we haven't included the permutation. Need to understand why there are $d$ fixed points. I should look into purity and energies of each of the fixed points.

In [112]:
L3, U3 = np.linalg.eig(P_E)
assert np.isclose(np.linalg.norm(P_E - U3 @ np.diag(L3) @ sp.linalg.inv(U3)), 0.0, atol=1.e-7, rtol=1.e-7)

For the non-Hermitian matrices, we need to use the inverse rather than the Hermitian conjugate.

In [113]:
print(np.max(L3))
L3_index = np.where(np.isclose(np.abs(L3), 1.0, atol=1.e-14, rtol=1.e-14))[0]
print(L3_index, len(L3_index))

(0.9999999999999966+1.4781126342604455e-15j)
[52] 1


Now, with the permutation inserted, we have a unique fixed point (at least for our random Hamiltonian)! The permutation is doing something.

In [114]:
L3_sort = np.abs(L3)
L3_sort.sort()
print(L3_sort[-len(L3_index)-1:])

[0.98565702 1.        ]


We have a spectral gap!

In [115]:
L4, U4 = np.linalg.eigh(M_E)
assert np.isclose(np.linalg.norm(M_E - U4 @ np.diag(L4) @ U4.conj().T), 0.0)
#assert np.isclose(np.linalg.norm(E - U2 @ np.diag(L2) @ sp.linalg.inv(U2)), 0.0)

In [116]:
print(np.max(L4))
L4_index = np.where(np.isclose(np.abs(L4), 1.0, atol=1.e-14, rtol=1.e-14))[0]
print(L4_index, len(L4_index))

1.0000000000000007
[90 91 92 93 94 95 96 97 98 99] 10


In [117]:
L4_sort = np.abs(L4)
L4_sort.sort()
print(L4_sort[-len(L4_index)-1:])

[0.99988036 1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.        ]


# Investigate fixed points

## Permutation channel

In [56]:
L_fp = []
valid_fps = []
for i in L3_index:
    L_fp.append(L3[i])
    fp = U3[:,i].reshape(d,d)
    assert np.isclose(np.linalg.norm(fp - fp.conj().T), 0.0)
    fp = fp + fp.conj().T
    fp /= fp.trace()
    print(i, L3[i], np.linalg.eigh(fp)[0])
    valid_fps.append(fp)

52 (0.9999999999999966+1.4781126342604455e-15j) [-2.18988265e-15 -9.42923396e-16  2.41049078e-16  5.04781452e-16
  1.21606951e-15  2.70220446e-15  6.18755885e-15  8.68813756e-15
  1.03355022e-14  1.00000000e+00]


In [57]:
print("GROUND STATE ENERGY:", L[0])
for i, vfp in enumerate(valid_fps):
    print("Fixed point:", i)
    print("Purity:", np.trace(vfp @ vfp))
    print("Energy:", np.trace(H @ vfp))

GROUND STATE ENERGY: -2.4880328494621176
Fixed point: 0
Purity: (0.9999999999999464+0j)
Energy: (-2.4880328494619945+6.938893903907228e-18j)


For the permutation channel, the single fixed point is the ground state; it's a pure quantum state and has the ground state energy.

## Non-permutation channel
The eigenvectors form a basis for the fixed point subspace, but they themselves are not Hermitian or PSD.

In [74]:
L_fp = []
valid_fps = []
for i in L2_index:
    L_fp.append(L2[i])
    fp = U2[:,i].reshape(d,d)
    if np.linalg.norm(fp - fp.conj().T) < 1.e-4:
    #assert np.isclose(np.linalg.norm(fp - fp.conj().T), 0.0)
        fp = fp + fp.conj().T
        fp /= fp.trace()
        print(i, L2[i], np.linalg.eigh(fp)[0])
        valid_fps.append(fp)

97 1.0000000000000004 [0.00178329 0.00548876 0.01344164 0.03152824 0.03184695 0.05234559
 0.07833029 0.18448398 0.23196026 0.36879099]


In [75]:
print("GROUND STATE ENERGY:", L[0])
for i, vfp in enumerate(valid_fps):
    print("Fixed point:", i)
    print("Purity:", np.trace(vfp @ vfp))
    print("Energy:", np.trace(H @ vfp))

GROUND STATE ENERGY: -2.4880328494621176
Fixed point: 0
Purity: (0.2349446369533869+0j)
Energy: (1.7845605593294618-2.168404344971009e-17j)


## Weak measurement channel
The eigenvectors form a basis for the fixed point subspace, but they themselves are not Hermitian or PSD.

In [71]:
L_fp = []
valid_fps = []
for i in L4_index:
    L_fp.append(L4[i])
    fp = U4[:,i].reshape(d,d)
    if np.linalg.norm(fp - fp.conj().T) < 1.e-4:
    #assert np.isclose(np.linalg.norm(fp - fp.conj().T), 0.0)
        fp = fp + fp.conj().T
        fp /= fp.trace()
        print(i, L4[i], np.linalg.eigh(fp)[0])
        valid_fps.append(fp)

97 1.0000000000000002 [0.00178329 0.00548876 0.01344164 0.03152825 0.03184695 0.05234559
 0.07833029 0.18448398 0.23196026 0.36879099]


In [72]:
print("GROUND STATE ENERGY:", L[0])
for i, vfp in enumerate(valid_fps):
    print("Fixed point:", i)
    print("Purity:", np.trace(vfp @ vfp))
    print("Energy:", np.trace(H @ vfp))

GROUND STATE ENERGY: -2.4880328494621176
Fixed point: 0
Purity: (0.23494463483966113+0j)
Energy: (1.7845605770227526+1.3660947373317356e-17j)
